In [25]:
from collections import deque
import gym
import matplotlib.pyplot as plt
from IPython import display
import time
import torch
import torch.nn as nn
import numpy as np
import random
%matplotlib inline

In [26]:
env = gym.make('CartPole-v0')
state = env.reset()
for i in range(1):
#     plt.imshow(env.render(mode='rgb_array'))
#     display.display(plt.gcf())
#     display.clear_output(wait=True)

#     env.render()
    action = env.action_space.sample()
    state,reward,done,info = env.step(action)
    print(state.shape)
#     print(f'{done}/action={action}')
    print(reward)
    time.sleep(0.02)
    if done:
        print("finish")
        break
# time.sleep(5)
# env.close()

(4,)
1.0


In [27]:
torch.from_numpy(state.reshape(1,4))

tensor([[ 0.0376, -0.1587, -0.0397,  0.2723]])

In [28]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
epochs = 100

In [29]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet,self).__init__()
        self.lin1 = nn.Linear(state_size,32)
        self.lin2 = nn.Linear(32,32)
        self.lin3 = nn.Linear(32,action_size)
        self.relu = nn.ReLU()
    def forward(self,x):
        pred = torch.from_numpy(x.reshape(1,state_size)).float()
        pred = self.lin1(pred)
        pred = self.relu(pred)
        pred = self.lin2(pred)
        pred = self.relu(pred)
        pred = self.lin3(pred)
        return pred

In [30]:
class dqn_agent():
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1
        self.epsilon_rate = 0.995
        self.epsilon_lim = 0.01
        self.lr = 0.001
        self.model = NNet()
        self.criterion = nn.MSELoss()
        self.optim = torch.optim.Adam(self.model.parameters(),lr=self.lr)
    def train(self,batch_size):
        batch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in batch:
            pred = self.model(state)[0][action]
            target = torch.tensor(reward,dtype=torch.float32)
            if not done:
                target = target+torch.max(self.model(next_state)[0])
            loss = self.criterion(pred,target)
            loss.backward()
            self.optim.step()
            self.optim.zero_grad()
        if self.epsilon>self.epsilon_lim:
            self.epsilon*=self.epsilon_rate
    def action(self,state):
        if np.random.rand()<=self.epsilon:
            return torch.tensor(random.randrange(self.action_size))
        else:
            return torch.argmax(self.model(state)[0])
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))

In [31]:
env = gym.make('CartPole-v0')

In [33]:
agent = dqn_agent(state_size,action_size)

for epoch in range(epochs):
    state = env.reset()
    
    done = False
    times = 0
    while not done:
        action = agent.action(state)
        if epoch%20 == 0:
            env.render()
        if epoch%20 == 0:
            time.sleep(0.02)
        next_state, reward, done, info = env.step(action.numpy())
        reward = reward if not done else -100
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        if done:
            print(f'epoch = {epoch}/{epochs}, score = {times}, epsilon = {agent.epsilon:.4f}')
        times+=1
        if len(agent.memory)>=batch_size:
            agent.train(batch_size)
    if epoch%20 == 0:
        env.close()
        

epoch = 0/100, score = 35, epsilon = 0.9801
epoch = 1/100, score = 8, epsilon = 0.9369
epoch = 2/100, score = 109, epsilon = 0.5398
epoch = 3/100, score = 59, epsilon = 0.3996
epoch = 4/100, score = 41, epsilon = 0.3237
epoch = 5/100, score = 35, epsilon = 0.2703
epoch = 6/100, score = 36, epsilon = 0.2245
epoch = 7/100, score = 27, epsilon = 0.1951
epoch = 8/100, score = 44, epsilon = 0.1557
epoch = 9/100, score = 38, epsilon = 0.1281
epoch = 10/100, score = 135, epsilon = 0.0648
epoch = 11/100, score = 77, epsilon = 0.0438
epoch = 12/100, score = 109, epsilon = 0.0252
epoch = 13/100, score = 31, epsilon = 0.0215
epoch = 14/100, score = 73, epsilon = 0.0148
epoch = 15/100, score = 89, epsilon = 0.0100
epoch = 16/100, score = 94, epsilon = 0.0100
epoch = 17/100, score = 92, epsilon = 0.0100
epoch = 18/100, score = 64, epsilon = 0.0100
epoch = 19/100, score = 42, epsilon = 0.0100
epoch = 20/100, score = 39, epsilon = 0.0100
epoch = 21/100, score = 129, epsilon = 0.0100
epoch = 22/100, s

KeyboardInterrupt: 